**This notebook is an exercise in the [Advanced SQL](https://www.kaggle.com/learn/advanced-sql) course.  You can reference the tutorial at [this link](https://www.kaggle.com/alexisbcook/analytic-functions).**

---


# Introduction

Here, you'll use window functions to answer questions about the [Chicago Taxi Trips](https://www.kaggle.com/chicago/chicago-taxi-trips-bq) dataset.

Before you get started, run the code cell below to set everything up.

In [1]:
# Get most recent checking code
!pip install -U -t /kaggle/working/ git+https://github.com/Kaggle/learntools.git
# Set up feedback system
from learntools.core import binder
binder.bind(globals())
from learntools.sql_advanced.ex2 import *
print("Setup Complete")

  Cloning https://github.com/Kaggle/learntools.git to /tmp/pip-req-build-70wqaxtq
  Running command git clone --filter=blob:none --quiet https://github.com/Kaggle/learntools.git /tmp/pip-req-build-70wqaxtq
  fatal: unable to access 'https://github.com/Kaggle/learntools.git/': Could not resolve host: github.com
  error: subprocess-exited-with-error
  
  × git clone --filter=blob:none --quiet https://github.com/Kaggle/learntools.git /tmp/pip-req-build-70wqaxtq did not run successfully.
  │ exit code: 128
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
error: subprocess-exited-with-error

× git clone --filter=blob:none --quiet https://github.com/Kaggle/learntools.git /tmp/pip-req-build-70wqaxtq did not run successfully.
│ exit code: 128
╰─> See above for output.

note: This error originates from a subprocess, and is likely not a problem with pip.
Using Kaggle's public dataset BigQuery integration.


/opt/conda/lib/python3.7/site-packages/google/cloud/bigquery/client.py:440: UserWarning: Cannot create BigQuery Storage client, the dependency google-cloud-bigquery-storage is not installed.
  "Cannot create BigQuery Storage client, the dependency "


Setup Complete


The following code cell fetches the `taxi_trips` table from the `chicago_taxi_trips` dataset. We also preview the first five rows of the table.  You'll use the table to answer the questions below.

In [2]:
from google.cloud import bigquery

# Create a "Client" object
client = bigquery.Client()

# Construct a reference to the "chicago_taxi_trips" dataset
dataset_ref = client.dataset("chicago_taxi_trips", project="bigquery-public-data")

# API request - fetch the dataset
dataset = client.get_dataset(dataset_ref)

# Construct a reference to the "taxi_trips" table
table_ref = dataset_ref.table("taxi_trips")

# API request - fetch the table
table = client.get_table(table_ref)

# Preview the first five lines of the table
client.list_rows(table, max_results=5).to_dataframe()

Using Kaggle's public dataset BigQuery integration.


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:19: UserWarning: Cannot use bqstorage_client if max_results is set, reverting to fetching data with the tabledata.list endpoint.


,unique_key,taxi_id,trip_start_timestamp,trip_end_timestamp,trip_seconds,trip_miles,pickup_census_tract,dropoff_census_tract,pickup_community_area,dropoff_community_area,...,extras,trip_total,payment_type,company,pickup_latitude,pickup_longitude,pickup_location,dropoff_latitude,dropoff_longitude,dropoff_location
0,0523e3d9d670b1730853950fa33f3ca42cfe9e35,7c3d71cce1e7d945d5d1736ff35a8f3c05669d8106df27...,2014-11-03 16:00:00+00:00,2014-11-03 16:30:00+00:00,1920,0.6,1.703184e+10,1.703198e+10,32.0,56.0,...,3.0,28.05,Cash,Taxi Affiliation Services,41.880994,-87.632746,POINT (-87.6327464887 41.8809944707),41.785999,-87.750934,POINT (-87.7509342894 41.785998518)
1,89b7e0e6d6e3579fc2f00a430aabc3365c1a8043,c9945e7e18aa8663c2619baf25d0df8730793b96b5003f...,2014-11-20 13:15:00+00:00,2014-11-20 13:45:00+00:00,2040,0.0,1.703132e+10,1.703198e+10,32.0,76.0,...,6.0,44.45,Cash,Dispatch Taxi Affiliation,41.884987,-87.620993,POINT (-87.6209929134 41.8849871918),41.979071,-87.903040,POINT (-87.9030396611 41.9790708201)
2,33c8c13f8d630ac734672db3d50bc55d78a9c366,da4802fcc8e9eb10ee00b1212582430f9a6af7bc2982f9...,2014-11-02 02:30:00+00:00,2014-11-02 02:30:00+00:00,0,0.0,NaN,NaN,NaN,NaN,...,0.0,16.14,Credit Card,Chicago Elite Cab Corp. (Chicago Carriag,NaN,NaN,None,NaN,NaN,None
3,e5983cc70ed90161180145340c52dfe9d10d1fdb,2509c81056c1e0cb62c7506298a354992d45550bba9fd0...,2014-11-20 13:15:00+00:00,2014-11-20 13:45:00+00:00,1920,12.4,1.703108e+10,1.703198e+10,8.0,56.0,...,3.0,37.25,Credit Card,Taxi Affiliation Services,41.892508,-87.626215,POINT (-87.6262149064 41.8925077809),41.785999,-87.750934,POINT (-87.7509342894 41.785998518)
4,47cee9258b9b4acc51191761018d8d2d5f6115cd,4dd2c499779e1efe86d8e7484cbccafef02bac165c79e7...,2014-11-07 19:15:00+00:00,2014-11-07 19:15:00+00:00,420,0.0,1.703108e+10,1.703108e+10,8.0,8.0,...,0.0,6.05,Cash,Taxi Affiliation Services,41.895033,-87.619711,POINT (-87.6197106717 41.8950334495),41.892042,-87.631864,POINT (-87.6318639497 41.8920421365)


# Exercises

### 1) How can you predict the demand for taxis?

Say you work for a taxi company, and you're interested in predicting the demand for taxis.  Towards this goal, you'd like to create a plot that shows a rolling average of the daily number of taxi trips.  Amend the (partial) query below to return a DataFrame with two columns:
- `trip_date` - contains one entry for each date from January 1, 2016, to March 31, 2016.
- `avg_num_trips` - shows the average number of daily trips, calculated over a window including the value for the current date, along with the values for the preceding 3 days and the following 3 days, as long as the days fit within the three-month time frame.  For instance, when calculating the value in this column for January 3, 2016, the window will include the number of trips for the preceding 2 days, the current date, and the following 3 days.

This query is partially completed for you, and you need only write the part that calculates the `avg_num_trips` column.  Note that this query uses a common table expression (CTE); if you need to review how to use CTEs, you're encouraged to check out [this tutorial](https://www.kaggle.com/dansbecker/as-with) in the [Intro to SQL](https://www.kaggle.com/learn/intro-to-sql) course.

In [3]:
# Fill in the blank below
avg_num_trips_query = """
                      WITH trips_by_day AS
                      (
                      SELECT DATE(trip_start_timestamp) AS trip_date,
                          COUNT(*) as num_trips
                      FROM `bigquery-public-data.chicago_taxi_trips.taxi_trips`
                      WHERE trip_start_timestamp > '2016-01-01' AND trip_start_timestamp < '2016-04-01'
                      GROUP BY trip_date
                      ORDER BY trip_date
                      )
                      SELECT trip_date,
                          AVG(num_trips)
                          OVER (
                               ORDER BY trip_date
                               ROWS BETWEEN 3 PRECEDING AND 3 FOLLOWING
                               ) AS avg_num_trips
                      FROM trips_by_day
                      """

# Check your answer
q_1.check()

/opt/conda/lib/python3.7/site-packages/google/cloud/bigquery/client.py:440: UserWarning: Cannot create BigQuery Storage client, the dependency google-cloud-bigquery-storage is not installed.
  "Cannot create BigQuery Storage client, the dependency "


,trip_date,avg_num_trips
0,2016-02-13,31744.714286
1,2016-01-16,34978.857143
2,2016-01-28,28300.142857
3,2016-01-15,35234.142857
4,2016-03-07,24254.714286


<IPython.core.display.Javascript object>

<span style="color:#33cc33">Correct</span>

In [4]:
# Lines below will give you a hint or solution code
q_1.hint()
q_1.solution()

<IPython.core.display.Javascript object>

<span style="color:#3366cc">Hint:</span> Use the **AVG()** function. Write an **OVER** clause with that orders the rows with the `trip_date` column and uses a window that includes the 3 preceding rows, the current row, and the following 3 rows.

<IPython.core.display.Javascript object>

<span style="color:#33cc99">Solution:</span> 
```python

avg_num_trips_query = """
                      WITH trips_by_day AS
                      (
                      SELECT DATE(trip_start_timestamp) AS trip_date,
                          COUNT(*) as num_trips
                      FROM `bigquery-public-data.chicago_taxi_trips.taxi_trips`
                      WHERE trip_start_timestamp > '2016-01-01' AND trip_start_timestamp < '2016-04-01'
                      GROUP BY trip_date
                      )
                      SELECT trip_date,
                          AVG(num_trips) 
                          OVER (
                               ORDER BY trip_date
                               ROWS BETWEEN 3 PRECEDING AND 3 FOLLOWING
                               ) AS avg_num_trips
                      FROM trips_by_day
                      """

```

### 2) Can you separate and order trips by community area?

The query below returns a DataFrame with three columns from the table: `pickup_community_area`, `trip_start_timestamp`, and `trip_end_timestamp`.  

Amend the query to return an additional column called `trip_number` which shows the order in which the trips were taken from their respective community areas.  So, the first trip of the day originating from community area 1 should receive a value of 1; the second trip of the day from the same area should receive a value of 2.  Likewise, the first trip of the day from community area 2 should receive a value of 1, and so on.

Note that there are many numbering functions that can be used to solve this problem (depending on how you want to deal with trips that started at the same time from the same community area); to answer this question, please use the **RANK()** function.

In [5]:
# Amend the query below
trip_number_query = """
                    SELECT pickup_community_area,
                        trip_start_timestamp,
                        trip_end_timestamp,
                        RANK() 
                            OVER (
                                PARTITION BY pickup_community_area
                                ORDER BY trip_start_timestamp
                                ) as trip_number
                    FROM `bigquery-public-data.chicago_taxi_trips.taxi_trips`
                    WHERE DATE(trip_start_timestamp) = '2013-10-03'
                    """

# Check your answer
q_2.check()

/opt/conda/lib/python3.7/site-packages/google/cloud/bigquery/client.py:440: UserWarning: Cannot create BigQuery Storage client, the dependency google-cloud-bigquery-storage is not installed.
  "Cannot create BigQuery Storage client, the dependency "


,pickup_community_area,trip_start_timestamp,trip_end_timestamp,trip_number
0,23.0,2013-10-03 04:15:00+00:00,2013-10-03 04:30:00+00:00,1
1,23.0,2013-10-03 11:30:00+00:00,2013-10-03 11:30:00+00:00,2
2,23.0,2013-10-03 13:15:00+00:00,2013-10-03 13:15:00+00:00,3
3,23.0,2013-10-03 13:45:00+00:00,2013-10-03 14:00:00+00:00,4
4,23.0,2013-10-03 16:00:00+00:00,2013-10-03 16:00:00+00:00,5


<IPython.core.display.Javascript object>

<span style="color:#33cc33">Correct</span>

In [6]:
# Lines below will give you a hint or solution code
q_2.hint()
q_2.solution()

<IPython.core.display.Javascript object>

<span style="color:#3366cc">Hint:</span> You don't need a **GROUP BY** clause.  Use the **RANK()** function.  Your **OVER** clause should order the rows by the `trip_start_timestamp` column and break the data into partitions based on `pickup_community_area`.

<IPython.core.display.Javascript object>

<span style="color:#33cc99">Solution:</span> 
```python

trip_number_query = """
                    SELECT pickup_community_area,
                        trip_start_timestamp,
                        trip_end_timestamp,
                        RANK()
                            OVER (
                                  PARTITION BY pickup_community_area
                                  ORDER BY trip_start_timestamp
                                 ) AS trip_number
                    FROM `bigquery-public-data.chicago_taxi_trips.taxi_trips`
                    WHERE DATE(trip_start_timestamp) = '2013-10-03' 
                    """

trip_number_result = client.query(trip_number_query).result().to_dataframe()

```

### 3) How much time elapses between trips?

The (partial) query in the code cell below shows, for each trip in the selected time frame, the corresponding `taxi_id`, `trip_start_timestamp`, and `trip_end_timestamp`. 

Your task in this exercise is to edit the query to include an additional `prev_break` column that shows the length of the break (in minutes) that the driver had before each trip started (this corresponds to the time between `trip_start_timestamp` of the current trip and `trip_end_timestamp` of the previous trip).  Partition the calculation by `taxi_id`, and order the results within each partition by `trip_start_timestamp`.

Some sample results are shown below, where all rows correspond to the same driver (or `taxi_id`).  Take the time now to make sure that the values in the `prev_break` column make sense to you!

![first_commands](https://i.imgur.com/zzHgmlx.png)

Note that the first trip of the day for each driver should have a value of **NaN** (not a number) in the `prev_break` column.

In [7]:
# Fill in the blanks below
break_time_query = """
                   SELECT taxi_id,
                       trip_start_timestamp,
                       trip_end_timestamp,
                       TIMESTAMP_DIFF(
                           trip_start_timestamp, 
                           LAG(trip_end_timestamp)
                               OVER (
                                    PARTITION BY taxi_id 
                                    ORDER BY trip_start_timestamp), 
                           MINUTE) as prev_break
                   FROM `bigquery-public-data.chicago_taxi_trips.taxi_trips`
                   WHERE DATE(trip_start_timestamp) = '2013-10-03' 
                   """

# Check your answer
q_3.check()

/opt/conda/lib/python3.7/site-packages/google/cloud/bigquery/client.py:440: UserWarning: Cannot create BigQuery Storage client, the dependency google-cloud-bigquery-storage is not installed.
  "Cannot create BigQuery Storage client, the dependency "


,taxi_id,trip_start_timestamp,trip_end_timestamp,prev_break
0,1eaef5df745de5ff990139aa7cce76faffa732d4579cac...,2013-10-03 00:00:00+00:00,2013-10-03 00:00:00+00:00,NaN
1,1eaef5df745de5ff990139aa7cce76faffa732d4579cac...,2013-10-03 01:15:00+00:00,2013-10-03 01:30:00+00:00,75.0
2,1eaef5df745de5ff990139aa7cce76faffa732d4579cac...,2013-10-03 02:15:00+00:00,2013-10-03 02:45:00+00:00,45.0
3,1ec7f3414307d2d25d1a35fa094f8591797f6bc2f0b1c9...,2013-10-03 02:30:00+00:00,2013-10-03 02:45:00+00:00,NaN
4,3df7b3266ecf9333cb3ed789437f20f9c556a6fe2df7be...,2013-10-03 05:45:00+00:00,2013-10-03 06:00:00+00:00,NaN


<IPython.core.display.Javascript object>

<span style="color:#33cc33">Correct</span>

In [8]:
# Lines below will give you a hint or solution code
q_3.hint()
#q_3.solution()

<IPython.core.display.Javascript object>

<span style="color:#3366cc">Hint:</span> The `TIMESTAMP_DIFF()` function takes three arguments, where the first (`trip_start_timestamp`) and the last (`MINUTE`) are provided for you.  This function provides the time difference (in minutes) of the timestamps in the first two arguments. You need only fill in the second argument, which should use the **LAG()** function to pull the timestamp corresponding to the end of the previous trip (for the same `taxi_id`).

# Keep going

Move on to learn how to query **[nested and repeated data](https://www.kaggle.com/alexisbcook/nested-and-repeated-data)**.

---




*Have questions or comments? Visit the [course discussion forum](https://www.kaggle.com/learn/advanced-sql/discussion) to chat with other learners.*